In [1]:
import  sqlalchemy as sa
from sqlalchemy import create_engine
import urllib
import pyodbc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

conn = urllib.parse.quote_plus(
      'Data Source Name = MssqlDataSource;'
      'Driver ={SQL Server};'
      'Server = NTBK630327\SQLEXPRESS;'
      'Database=Electronic_sales;'
      'Trusted_connection=yes;'
)

try:
    
    coxn = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(conn))
    print("Passed")
    
except:
    print("failed")

Passed


The first thing we need to do, before starting to query, it's connect to the database.

We can do that very easily using psycopg2 library adding the following line to the code:                                       
cnxn = pyodbc.connect('DSN=MssqlDataSource; DRIVER = {SQL Server} ; SERVER=NTBK630327\SQLEXPRESS; DATABASE = Electronic_sales; UID="";PWD=""')                                                                                                               
cursor =cnxn.cursor()

**Now we can generate dataframes from string queries using pandas .`read_sql like` so:**

In [2]:
Data_Source_Name = 'MssqlDataSource'
server = 'NTBK630327\SQLEXPRESS'
database = 'Electronic_sales'
username = ''
password = ''
cnxn  = pyodbc.connect('DSN='+Data_Source_Name+'; DRIVER = {SQL Server} ; SERVER=' +server+'; DATABASE=' +database+' ; UID=' +username+'; PWD=' + password)
cursor =cnxn.cursor()
query = "SELECT * FROM dbo.all_data;"
df = pd.read_sql(query , cnxn)

In [3]:
import pyodbc
import pandas as pd
#Data_Source_Name = 'mssqldatasource'
server = 'SQLEXPRESS'
database = 'Electronic_sales'
username = ''
password = ''
#cnxn  = pyodbc.connect(' DRIVER = {SQL Server} ; SERVER=' +server+'; DATABASE=' +database+
#                              ' ; UID=' +username+'; PWD=' + password+'; Trusted_Connection=yes')
cnxn = pyodbc.connect('DSN=MssqlDataSource; DRIVER = {SQL Server} ; SERVER=NTBK630327\SQLEXPRESS; DATABASE = Electronic_sales; UID="";PWD=""')
cursor =cnxn.cursor()
query = "SELECT * FROM Electronic_sales.dbo.all_data;"
df = pd.read_sql(query , cnxn)


### 1. Let's inspect the database

In [4]:
query = """
SELECT TABLE_NAME
FROM INFORMATION_SCHEMA.TABLES

"""

pd.read_sql( query , cnxn)

,TABLE_NAME
0,spt_fallback_db
1,spt_fallback_dev
2,spt_fallback_usg
3,percentpopulationvaccinated
4,percentpopulationvaccinated2
5,percentpopulationvaccinated3
6,PercentPopulationVaccinated4
7,percentpopulationvaccinated7
8,daily_sale
9,spt_values


### 2.Print schemas

The table information_schema.columns contains schema information on each schema. We can query it to display schemas of all the public tables. For example, we could be interested in particular in the column names and data types. We can check that by selecting specifically the columns table_name, data_type, and table_schema

In [5]:
query = """
 SELECT TABLE_NAME , DATA_TYPE , TABLE_SCHEMA
 FROM INFORMATION_SCHEMA.COLUMNS
"""

pd.read_sql(query , cnxn)



,TABLE_NAME,DATA_TYPE,TABLE_SCHEMA
0,percentpopulationvaccinated7,nvarchar,dbo
1,percentpopulationvaccinated7,nvarchar,dbo
2,percentpopulationvaccinated7,datetime,dbo
3,percentpopulationvaccinated7,float,dbo
4,percentpopulationvaccinated7,nvarchar,dbo
...,...,...,...
78,spt_values,int,dbo
79,spt_values,int,dbo
80,daily_sale,datetime,dbo
81,daily_sale,time,dbo


Let's read the first three rows of the table all_data.

In [6]:
df.head(3)


,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,176558,USB-C Charging Cable,2,11.95,2019-04-19,"""917 1st St, Dallas, TX 75001"""
1,176559,Bose SoundSport Headphones,1,99.99,2019-04-07,"""682 Chestnut St, Boston, MA 02215"""
2,176560,Google Phone,1,600.00,2019-04-12,"""669 Spruce St, Los Angeles, CA 90001"""


In [7]:
# Converting all the columns name in order to remove all space between columns names
df.columns = [c.replace(' ', '_') for c in df.columns]

Inserting a dataframe in to a MS SQL server database table 

Firstly we create the table into the database  then we create a connection to the database and finaly we read all rows in our dataframe and insert into the table we created earlier.

In [8]:
"""
/*
MS SQL SERVER CODE 
*/
USE Electronic-sales
GO

DROP IF EXISTS sample_sales
CREATE TABLE sample_sales(
  [Order_ID] nvarchar(50),
  [Product]  nvarchar(50),
  [Quantity_Ordered] int,
  ['Price_Each']    float,
  ['Order_Date']  date,
  ['Purchase_Address']  nvarchar(50)
 )


"""



cnxn = pyodbc.connect('DSN=MssqlDataSource; DRIVER = {SQL Server} ; SERVER=NTBK630327\SQLEXPRESS; DATABASE = Electronic_sales; UID="";PWD=""')
cursor =cnxn.cursor()

for index,row in df.iterrows():
    cursor.execute("INSERT INTO Electronic_sales.dbo.sample_sales ([Order_ID],[Product],[Quantity_Ordered],[Price_Each],[Order_Date],[Purchase_Address]) values (?,?,?,?,?,?)",row['Order_ID_'],row['Product'],row['Quantity_Ordered'] ,row['Price_Each'],row['Order_Date'],row['Purchase_Address'])
    cnxn.commit()
    
cursor.close()
cnxn.close()
